In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip install mne

     |████████████████████████████████| 7.5 MB 12.0 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
# localized fetch_data function by using mne library
# https://github.com/mne-tools/mne-python/blob/maint/0.20/mne/datasets/sleep_physionet/age.py#L18-L111
data_path = _data_path
BASE_URL = 'https://physionet.org/physiobank/database/sleep-edfx/sleep-cassette/'
def fetch_data(subjects, recording=[1, 2], path=None, force_update=False,
               update_path=None, base_url=BASE_URL,
               verbose=None):  # noqa: D301
    records = np.loadtxt(AGE_SLEEP_RECORDS,
                         skiprows=1,
                         delimiter=',',
                         usecols=(0, 1, 2, 6, 7),
                         dtype={'names': ('subject', 'record', 'type', 'sha',
                                          'fname'),
                                'formats': ('<i2', 'i1', '<S9', 'S40', '<S22')}
                         )
    psg_records = records[np.where(records['type'] == b'PSG')]
    hyp_records = records[np.where(records['type'] == b'Hypnogram')]

    path = data_path(path=path, update_path=update_path)
    params = [path, force_update, base_url]
    fnames = []
    for subject in subjects:
        for idx in np.where(psg_records['subject'] == subject)[0]:
            if psg_records['record'][idx] in recording:
                psg_fname = _fetch_one(psg_records['fname'][idx].decode(),
                                       psg_records['sha'][idx].decode(),
                                       *params)
                hyp_fname = _fetch_one(hyp_records['fname'][idx].decode(),
                                       hyp_records['sha'][idx].decode(),
                                       *params)
                fnames.append([psg_fname, hyp_fname])

    return fnames

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
npz_files = sorted(glob.glob("/gdrive/My Drive/NPZ_files/***.npz"))
X = np.zeros((0, 3000, 1))
y = []
for fn in tqdm(npz_files[:number_of_files]):
    samples = np.load(fn)
    X_data = samples['x']
    X = np.concatenate((X, X_data), axis=0)
    y.extend(samples['y'])
y = np.array(y)

#sorted(glob.glob("/gdrive/My Drive/NPZ_files/***.npz"))


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
npz_files

['/gdrive/My Drive/NPZ_files/SC4001E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4002E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4011E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4012E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4021E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4022E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4031E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4032E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4041E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4042E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4051E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4052E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4061E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4062E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4071E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4072E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4081E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4082E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4091E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4092E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4101E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4102E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4

In [ ]:
len(npz_files[:number_of_files])

100

In [ ]:
npz_files[:number_of_files]

['/gdrive/My Drive/NPZ_files/SC4001E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4002E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4011E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4012E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4021E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4022E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4031E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4032E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4041E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4042E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4051E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4052E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4061E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4062E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4071E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4072E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4081E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4082E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4091E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4092E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4101E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4102E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4

In [ ]:
npz_files[148:150]

['/gdrive/My Drive/NPZ_files/SC4821G0.npz',
 '/gdrive/My Drive/NPZ_files/SC4822G0.npz']

In [ ]:
X

array([[[ 8.11135578e+00],
        [ 1.74886456e+01],
        [ 2.12395611e+01],
        ...,
        [-1.03619051e+01],
        [-1.11120882e+01],
        [-2.10989022e+00]],

       [[-1.07369967e+01],
        [-1.13934069e+01],
        [-4.45421267e+00],
        ...,
        [ 5.88424911e+01],
        [ 4.83399277e+01],
        [ 5.36849823e+01]],

       [[ 6.13743591e+01],
        [ 3.86813202e+01],
        [ 4.95589752e+01],
        ...,
        [ 3.36175842e+01],
        [ 3.43677673e+01],
        [ 3.18358974e+01]],

       ...,

       [[ 3.47106218e+00],
        [-5.03052510e-02],
        [ 2.56556773e+00],
        ...,
        [ 9.55799758e-01],
        [-1.86129427e+00],
        [ 3.37045169e+00]],

       [[ 2.26373625e+00],
        [ 3.52136761e-01],
        [ 2.86739922e+00],
        ...,
        [ 2.54041519e+01],
        [ 2.02730160e+01],
        [ 1.82608051e+01]],

       [[ 1.96693535e+01],
        [ 1.65504265e+01],
        [ 1.54437122e+01],
        ...,
        

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
from keras.utils import np_utils

In [ ]:
if VBS:
    print("Shape of the input data: {}".format(X.shape))
    print("Shape of the sleep stages: {}".format(y.shape))


Shape of the input data: (116718, 3000, 1)
Shape of the sleep stages: (116718,)


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
def butter_bandpass(lowcut, highpass, fs, order=4):
    nyq = 0.5 * fs
    #       low = lowcut / nyq
    high = highpass / nyq
    b, a = butter(order, high, btype='highpass')
    return b, a

In [ ]:
def butter_bandpass_filter(data, highpass, fs, order=4):
    b, a = butter_bandpass(0, highpass, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
#npz_files = sorted(glob.glob(os.path.join(output_path, "*.npz")))
X_sub_482 = np.zeros((0, 3000, 1))
y_sub_482 = []
for fn in tqdm(npz_files[148:150]): #extracting data from the npz files corresponding to both tge recording 1, 2 of the 50th subject (no 53rd subject)
    samples = np.load(fn)
    X_data = samples['x']
    X_sub_482 = np.concatenate((X_sub_482, X_data), axis=0)
    y_sub_482.extend(samples['y'])
y_sub_482 = np.array(y_sub_482)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
X_sub_482

array([[[-49.10647202],
        [-48.57606888],
        [-45.21685028],
        ...,
        [  1.81220996],
        [ -2.51941395],
        [  3.75702071]],

       [[ 17.63589668],
        [  4.37582397],
        [ 25.59194183],
        ...,
        [ 52.20048904],
        [ 55.55970764],
        [ 57.06251526]],

       [[ 50.25567627],
        [ 55.64810562],
        [ 50.16727829],
        ...,
        [-13.03907204],
        [-16.75189209],
        [-13.12747288]],

       ...,

       [[  4.04444456],
        [  2.26666665],
        [  6.44444466],
        ...,
        [ -6.17777777],
        [ 10.17777824],
        [-48.93333435]],

       [[-43.95555496],
        [-63.42222214],
        [-70.8888855 ],
        ...,
        [ 20.66666603],
        [ 14.88888931],
        [ 22.35555649]],

       [[ 21.73333359],
        [ 21.55555534],
        [ 21.91111183],
        ...,
        [  1.28888893],
        [  2.97777772],
        [  5.28888893]]])

In [ ]:
y_sub_482

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
if VBS:
    print("Shape of the subject(482) data: {}".format(X_sub_482.shape))
    print("Shape of the corresponding sleep stage: {}".format(y_sub_482.shape))

Shape of the subject(482) data: (3070, 3000, 1)
Shape of the corresponding sleep stage: (3070,)


In [ ]:
pp_x = np.array([butter_bandpass_filter(sample, highpass=40.0, fs=100, order=4) for sample in X])

if VBS:
    print(pp_x.shape)

(116718, 3000, 1)


In [ ]:
y.shape

(116718,)

In [ ]:
type(y)

numpy.ndarray

In [ ]:
pp_x

array([[[ 3.91319654e-02],
        [ 8.43712310e-02],
        [ 1.02466935e-01],
        ...,
        [-4.99893880e-02],
        [-5.36085289e-02],
        [-1.01788349e-02]],

       [[-5.17989585e-02],
        [-5.49657067e-02],
        [-2.14886513e-02],
        ...,
        [ 2.83876381e-01],
        [ 2.33208409e-01],
        [ 2.58994788e-01]],

       [[ 2.96090982e-01],
        [ 1.86611970e-01],
        [ 2.39089513e-01],
        ...,
        [ 1.62182769e-01],
        [ 1.65801910e-01],
        [ 1.53587300e-01]],

       ...,

       [[ 1.67455958e-02],
        [-2.42689804e-04],
        [ 1.23771796e-02],
        ...,
        [ 4.61110622e-03],
        [-8.97952265e-03],
        [ 1.62602162e-02]],

       [[ 1.09210409e-02],
        [ 1.69882864e-03],
        [ 1.38333184e-02],
        ...,
        [ 1.22558352e-01],
        [ 9.78039900e-02],
        [ 8.80963939e-02]],

       [[ 9.48917148e-02],
        [ 7.98449401e-02],
        [ 7.45057705e-02],
        ...,
        

In [ ]:
pp_x_rshp=pp_x.reshape(116718, 3000)

In [ ]:
pp_x_rshp_df=pd.DataFrame(pp_x_rshp)

In [ ]:
pp_x_rshp_df['sleep_stage']=y

In [ ]:
pp_x_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,0.039132,0.084371,0.102467,0.033251,0.045465,0.026465,0.008369,0.001131,-0.006560,0.009274,0.014250,0.022393,0.032346,0.012893,-0.001131,0.002488,0.014250,0.016060,-0.007917,-0.029179,-0.005655,-0.004750,-0.031441,-0.062656,-0.039132,-0.036418,-0.068537,-0.028275,-0.024655,-0.060394,-0.079847,-0.056323,-0.058132,-0.052704,-0.078943,-0.064014,-0.026917,-0.046370,-0.021941,-0.024203,...,-0.060847,-0.072157,-0.041846,-0.006107,-0.024203,-0.041846,-0.041846,0.011536,-0.029179,-0.034608,-0.052251,-0.030989,-0.072609,-0.046370,-0.013346,-0.064918,-0.052251,-0.065371,-0.061299,-0.062204,-0.060394,-0.002941,-0.030537,-0.049085,-0.016965,-0.031441,-0.001583,-0.040037,-0.047275,-0.064014,-0.034156,0.001583,-0.057228,-0.041394,-0.023751,0.002488,-0.049989,-0.053609,-0.010179,0
1,-0.051799,-0.054966,-0.021489,-0.001131,-0.016512,-0.114229,-0.056775,-0.085728,-0.113324,-0.078490,-0.050894,-0.051799,-0.083919,-0.059942,-0.064014,-0.054513,-0.058132,0.014250,-0.020131,0.000679,-0.024203,-0.057228,-0.007012,-0.074419,-0.002036,-0.078490,-0.034156,-0.035965,0.007464,-0.007464,-0.024655,0.013798,-0.007012,-0.028275,-0.007012,-0.030989,-0.007464,-0.026465,0.003845,0.046823,...,0.233208,0.188874,0.187969,0.217827,0.143635,0.224613,0.137301,0.191588,0.216922,0.191136,0.163088,0.206517,0.210136,0.128253,0.163540,0.126444,0.154944,0.154944,0.197922,0.206970,0.216922,0.194303,0.187517,0.183898,0.220994,0.242256,0.201993,0.196112,0.280257,0.260352,0.264876,0.197922,0.267138,0.264876,0.312377,0.251304,0.283876,0.233208,0.258995,0
2,0.296091,0.186612,0.239090,0.259900,0.302877,0.316901,0.272567,0.281614,0.220541,0.265328,0.222803,0.214208,0.140920,0.119658,0.040037,0.084824,0.027822,0.106086,0.065823,0.029632,0.003845,-0.008369,-0.042299,-0.088895,-0.108800,-0.125087,-0.071252,-0.050442,-0.051799,-0.044108,-0.002941,-0.065823,0.011084,-0.144087,-0.082109,-0.157659,-0.102467,-0.075776,-0.040489,-0.049989,...,-0.292472,-0.290210,-0.287496,-0.278900,-0.269852,-0.251757,-0.232304,-0.209232,-0.159468,-0.101562,-0.076681,-0.028727,-0.022393,0.025108,0.066728,0.099753,0.081204,0.130063,0.168516,0.180731,0.187064,0.198827,0.177564,0.185255,0.197017,0.231851,0.224161,0.241804,0.227327,0.187517,0.164445,0.146349,0.176207,0.176659,0.149516,0.163540,0.162183,0.165802,0.153587,0
3,0.134134,0.116491,0.116491,0.070347,0.066728,0.037322,0.030989,0.039584,0.023751,0.020131,-0.006107,-0.003393,-0.010631,-0.007464,-0.050894,-0.029632,0.006107,-0.018774,0.021941,0.006107,0.016060,0.014250,-0.020131,0.014250,-0.001583,-0.029179,-0.039132,0.041846,-0.112420,-0.025108,-0.010179,0.013346,0.031894,0.007012,-0.031441,-0.010631,-0.026465,-0.015155,-0.005655,-0.018322,...,-0.189779,-0.235470,-0.254923,-0.281162,-0.270757,-0.300163,-0.300163,-0.291115,-0.282067,-0.280257,-0.270757,-0.259900,-0.251304,-0.238637,-0.238637,-0.234113,-0.224613,-0.208779,-0.199731,-0.202446,-0.195660,-0.191588,-0.177564,-0.140468,-0.157659,-0.152683,-0.155849,-0.144087,-0.124182,-0.100657,-0.087086,-0.040942,0.009726,0.035060,0.129610,0.045465,0.126896,0.205612,0.274829,0
4,0.333640,0.359878,0.381141,0.386117,0.373450,0.372998,0.360783,0.367117,0.359878,0.382046,0.351283,0.387022,0.396522,0.361688,0.324592,0.347664,0.241804,0.263519,0.259447,0.206065,0.225970,0.163088,0.125539,0.238637,0.009726,0.205612,0.039132,-0.016060,0.003393,-0.053609,-0.000226,0.359426,0.334997,0.049537,-0.149516,0.057228,0.016060,-0.051347,-0.035965,0.048632,...,-0.060847,-0.064014,-0.085276,-0.155397,-0.178017,-0.152230,-0.112872,-0.124634,-0.081204,-0.028275,-0.050442,-0.062204,-0.092967,-0.075323,-0.036870,-0.109705,-0.088443,-0.143182,-0.242709,-0.293377,-0.351735,-0.342687,-0.359426,-0.343592,-0.339521,-0.

In [ ]:
pp_x_rshp_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/original_50_subs_dt_original_y.csv',header=None)

In [ ]:
pp_x_rshp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.039132,0.084371,0.102467,0.033251,0.045465,0.026465,0.008369,0.001131,-0.006560,0.009274,...,-0.034156,0.001583,-0.057228,-0.041394,-0.023751,0.002488,-0.049989,-0.053609,-0.010179,0
1,-0.051799,-0.054966,-0.021489,-0.001131,-0.016512,-0.114229,-0.056775,-0.085728,-0.113324,-0.078490,...,0.264876,0.197922,0.267138,0.264876,0.312377,0.251304,0.283876,0.233208,0.258995,0
2,0.296091,0.186612,0.239090,0.259900,0.302877,0.316901,0.272567,0.281614,0.220541,0.265328,...,0.164445,0.146349,0.176207,0.176659,0.149516,0.163540,0.162183,0.165802,0.153587,0
3,0.134134,0.116491,0.116491,0.070347,0.066728,0.037322,0.030989,0.039584,0.023751,0.020131,...,-0.087086,-0.040942,0.009726,0.035060,0.129610,0.045465,0.126896,0.205612,0.274829,0
4,0.333640,0.359878,0.381141,0.386117,0.373450,0.372998,0.360783,0.367117,0.359878,0.382046,...,-0.121920,-0.060847,0.000679,0.060394,0.045465,0.084371,0.057680,0.122372,0.207422,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.117219,0.233710,0.088582,0.195365,0.141488,0.136634,0.103629,0.213810,0.167699,0.141488,...,-0.108482,-0.110424,-0.112365,-0.064798,-0.171096,-0.115278,-0.044898,-0.099746,-0.122073,0
116714,-0.087611,-0.076447,-0.056061,-0.058488,-0.075962,-0.053149,-0.065769,-0.104114,-0.086155,-0.121102,...,0.034219,0.093921,0.097319,0.070623,0.069167,0.040529,0.058488,0.046839,0.049266,0
116715,0.016746,-0.000243,0.012377,0.020629,0.014319,-0.042956,-0.085669,-0.076447,-0.066740,-0.060915,...,0.031792,0.027909,0.014804,0.017231,0.020143,0.001213,0.004611,-0.008980,0.016260,0
116716,0.010921,0.001699,0.013833,-0.000243,0.008494,-0.000243,0.002670,-0.001699,-0.003640,0.011406,...,0.126927,0.115763,0.124014,0.130810,0.130810,0.124014,0.122558,0.097804,0.088096,0


In [ ]:
dt_shuf1=pp_x_rshp_df.sample(frac=1,replace=True, random_state=991)

In [ ]:
dt_shuf1

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
87982,-0.088735,-0.104907,-0.132443,-0.152111,-0.109278,-0.072563,-0.058577,-0.048961,-0.083053,-0.076060,...,-0.004816,0.005674,-0.010935,-0.013995,-0.031915,-0.002631,0.027965,0.047633,0.040203,3
68528,0.600975,0.594189,0.584011,0.579345,0.569166,0.521241,0.527603,0.526331,0.537782,0.505549,...,0.162437,0.172616,0.149290,0.107302,0.092034,0.062345,0.065738,0.092458,0.064042,0
37536,-0.034046,-0.019160,-0.031151,0.006478,-0.041076,0.005238,-0.006754,-0.003032,-0.011716,0.016816,...,0.079671,0.063543,0.043695,0.073881,0.109444,0.090835,0.099106,0.117714,0.104481,0
49751,-0.057419,0.039738,-0.062918,0.072734,-0.031296,0.052111,-0.072084,0.065401,-0.058335,0.064943,...,-0.210943,-0.027172,-0.199945,-0.016631,-0.211860,0.001242,-0.197653,-0.003799,-0.241190,0
55894,-0.095076,-0.096344,-0.092538,-0.090000,-0.103957,-0.093807,-0.101843,-0.109878,-0.104380,-0.090423,...,0.042803,0.041534,0.038151,0.025039,0.038151,0.045764,0.062681,0.065219,0.076638,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66766,0.052937,0.026246,0.023744,0.017488,-0.033809,-0.036312,-0.043818,-0.009620,-0.005033,-0.022966,...,-0.022132,0.001640,0.007062,0.003308,0.052103,0.062529,0.040008,0.028748,0.070036,4
42895,-0.049391,0.001550,-0.088372,-0.002879,-0.073754,-0.010410,-0.087486,-0.012182,-0.066667,-0.031229,...,0.040975,0.017054,0.055592,0.012182,0.050277,0.011296,0.059136,-0.002879,0.044518,2
7912,-0.019850,-0.035156,0.010284,0.041374,0.065768,0.088248,0.081552,0.098771,0.106902,0.093988,...,0.086813,0.081552,0.071507,0.031808,0.052375,0.053810,0.091118,0.068638,0.093031,2
56020,0.194850,0.157920,0.165215,0.203969,0.175246,0.200778,0.200322,0.205337,0.195306,0.163392,...,-0.005757,0.061720,0.006553,0.101385,-0.037216,0.051233,-0.003478,0.004273,0.003817,0


In [ ]:
dt_shuf1['sleep_stage']

87982     3
68528     0
37536     0
49751     0
55894     1
         ..
66766     4
42895     2
7912      2
56020     0
114355    0
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_shuf1=dt_shuf1.drop('sleep_stage',axis=1)
y_shuf1=dt_shuf1['sleep_stage']

In [ ]:
x_shuf1=dt_shuf1.drop([3000],axis=1)
y_shuf1=dt_shuf1[3000]

In [ ]:
x_shuf1

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
87982,-0.088735,-0.104907,-0.132443,-0.152111,-0.109278,-0.072563,-0.058577,-0.048961,-0.083053,-0.076060,...,0.015289,-0.004816,0.005674,-0.010935,-0.013995,-0.031915,-0.002631,0.027965,0.047633,0.040203
68528,0.600975,0.594189,0.584011,0.579345,0.569166,0.521241,0.527603,0.526331,0.537782,0.505549,...,0.208666,0.162437,0.172616,0.149290,0.107302,0.092034,0.062345,0.065738,0.092458,0.064042
37536,-0.034046,-0.019160,-0.031151,0.006478,-0.041076,0.005238,-0.006754,-0.003032,-0.011716,0.016816,...,0.088768,0.079671,0.063543,0.043695,0.073881,0.109444,0.090835,0.099106,0.117714,0.104481
49751,-0.057419,0.039738,-0.062918,0.072734,-0.031296,0.052111,-0.072084,0.065401,-0.058335,0.064943,...,-0.022131,-0.210943,-0.027172,-0.199945,-0.016631,-0.211860,0.001242,-0.197653,-0.003799,-0.241190
55894,-0.095076,-0.096344,-0.092538,-0.090000,-0.103957,-0.093807,-0.101843,-0.109878,-0.104380,-0.090423,...,0.019541,0.042803,0.041534,0.038151,0.025039,0.038151,0.045764,0.062681,0.065219,0.076638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66766,0.052937,0.026246,0.023744,0.017488,-0.033809,-0.036312,-0.043818,-0.009620,-0.005033,-0.022966,...,0.001223,-0.022132,0.001640,0.007062,0.003308,0.052103,0.062529,0.040008,0.028748,0.070036
42895,-0.049391,0.001550,-0.088372,-0.002879,-0.073754,-0.010410,-0.087486,-0.012182,-0.066667,-0.031229,...,0.022370,0.040975,0.017054,0.055592,0.012182,0.050277,0.011296,0.059136,-0.002879,0.044518
7912,-0.019850,-0.035156,0.010284,0.041374,0.065768,0.088248,0.081552,0.098771,0.106902,0.093988,...,0.054767,0.086813,0.081552,0.071507,0.031808,0.052375,0.053810,0.091118,0.068638,0.093031
56020,0.194850,0.157920,0.165215,0.203969,0.175246,0.200778,0.200322,0.205337,0.195306,0.163392,...,-0.011684,-0.005757,0.061720,0.006553,0.101385,-0.037216,0.051233,-0.003478,0.004273,0.003817


In [ ]:
y_shuf1_=np_utils.to_categorical(y_shuf1)

In [ ]:
x_shuf1_rshp=np.array(x_shuf1).reshape(116718,3000,1)

In [ ]:
pp_X_sub_482 = np.array([butter_bandpass_filter(sample, highpass=40.0, fs=100, order=4) for sample in X_sub_482])

if VBS:
    print(pp_X_sub_482.shape)

(3070, 3000, 1)


In [ ]:
pp_X_sub_482

array([[[-0.23690648],
        [-0.23434764],
        [-0.21814161],
        ...,
        [ 0.00874272],
        [-0.01215452],
        [ 0.01812516]],

       [[ 0.08508162],
        [ 0.02111048],
        [ 0.12346431],
        ...,
        [ 0.25183308],
        [ 0.26803911],
        [ 0.27528917]],

       [[ 0.24245064],
        [ 0.26846557],
        [ 0.24202418],
        ...,
        [-0.06290496],
        [-0.08081688],
        [-0.06333144]],

       ...,

       [[ 0.01951179],
        [ 0.01093518],
        [ 0.03109021],
        ...,
        [-0.02980372],
        [ 0.0491011 ],
        [-0.23607121]],

       [[-0.21205669],
        [-0.30597058],
        [-0.34199232],
        ...,
        [ 0.09970309],
        [ 0.07182911],
        [ 0.10785088]],

       [[ 0.10484906],
        [ 0.1039914 ],
        [ 0.10570673],
        ...,
        [ 0.00621804],
        [ 0.01436582],
        [ 0.02551542]]])

The following corresponds to reshaping the existing 3d data to 2d and converting it to .csv. The csv file was created on google drive and later downloaded to the local pc

In [ ]:
pp_x_rshp=pp_x.reshape(189560,3000)

In [ ]:
pp_x_rshp

array([[ 0.03913197,  0.08437123,  0.10246694, ..., -0.04998939,
        -0.05360853, -0.01017883],
       [-0.05179896, -0.05496571, -0.02148865, ...,  0.28387638,
         0.23320841,  0.25899479],
       [ 0.29609098,  0.18661197,  0.23908951, ...,  0.16218277,
         0.16580191,  0.1535873 ],
       ...,
       [-0.03455384,  0.00189911,  0.05185315, ..., -0.0525553 ,
         0.025301  ,  0.01360005],
       [-0.00620155,  0.02170071, -0.06515632, ...,  0.0352018 ,
         0.0203506 ,  0.0437525 ],
       [ 0.12205883, -0.08855822, -0.07505712, ..., -0.06470629,
        -0.08810818, -0.10835982]])

In [ ]:
pp_X_sub_482_rshp=pp_X_sub_482.reshape(3070,3000)

In [ ]:
pp_X_sub_482_rshp_df=pd.DataFrame(pp_X_sub_482_rshp)

In [ ]:
pp_X_sub_482_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.236906,-0.234348,-0.218142,-0.187435,-0.143082,-0.184024,-0.207053,-0.213024,-0.187009,-0.258230,-0.239465,-0.255245,-0.197244,-0.247568,-0.243304,-0.288936,-0.264627,-0.252686,-0.224965,-0.231362,-0.326466,-0.266760,-0.224965,-0.231789,-0.321348,-0.260363,-0.269745,-0.236054,-0.228803,-0.259510,-0.398967,-0.433085,-0.476159,-0.439482,-0.430526,-0.419437,-0.386172,-0.360584,-0.346937,-0.335849,...,-0.112376,-0.044140,-0.023243,-0.036037,-0.033478,0.050537,0.065037,0.020684,0.055655,0.043714,0.129861,0.042008,0.088067,0.100861,0.161847,0.123891,0.052670,0.092332,0.002346,0.081670,0.092758,0.110244,0.033478,0.045420,0.120905,-0.000213,0.012581,0.042008,-0.021537,-0.005331,0.036464,0.134979,0.138817,0.049258,0.100861,0.046272,0.065037,0.008743,-0.012155,0.018125
1,0.085082,0.021110,0.123464,0.145215,0.168244,0.133273,0.203641,0.077832,0.107258,0.140097,0.189568,0.091905,0.181891,0.143082,0.151185,0.194686,0.251407,-0.022816,0.083376,0.144362,-0.019405,-0.000213,0.049258,0.088067,0.101714,-0.020684,-0.009169,0.031772,-0.000213,-0.021963,-0.059920,-0.009596,-0.009169,-0.003199,0.030066,0.053096,0.060773,0.017699,0.070155,0.114935,...,0.216862,0.193406,0.164406,0.147347,0.240745,0.167391,0.169097,0.219421,0.253539,0.282966,0.194259,0.225392,0.224112,0.277422,0.278274,0.297892,0.256951,0.268892,0.321348,0.268039,0.268039,0.404937,0.175494,0.184877,0.080817,-0.077405,-0.132420,-0.193833,-0.084655,-0.110670,-0.075699,0.056934,0.222833,0.234348,0.151185,0.167818,0.214303,0.251833,0.268039,0.275289
2,0.242451,0.268466,0.242024,0.272730,0.204494,0.171656,0.211318,0.180185,0.187009,0.182318,0.187862,0.150332,0.161421,0.206200,0.166965,0.145215,0.123891,0.088493,0.026228,0.054375,-0.031346,-0.094038,-0.346084,-0.320922,-0.319216,-0.333716,-0.317084,-0.302584,-0.288936,-0.277422,-0.275289,-0.259083,-0.239039,-0.212171,-0.205347,-0.173362,-0.151185,-0.117920,-0.089773,-0.041581,...,0.031346,0.037317,0.060773,0.040302,0.039875,0.056508,0.045420,0.048831,0.091905,0.109391,0.122185,0.059067,0.149479,0.115361,0.118347,0.133273,0.118347,0.085082,0.067170,0.071861,0.059067,0.063758,0.052243,-0.002772,-0.008743,-0.019831,0.008743,-0.045846,-0.171656,-0.112376,-0.143509,-0.120905,-0.083376,-0.061626,-0.100861,-0.088493,-0.078258,-0.062905,-0.080817,-0.063331
3,-0.134126,-0.107258,-0.096170,-0.097023,-0.117067,-0.099582,-0.099155,-0.100435,-0.083376,-0.098302,-0.118773,-0.105126,-0.098302,-0.044567,-0.040728,-0.050111,-0.065890,-0.069729,-0.038169,-0.020258,-0.083376,-0.065464,-0.065890,-0.008316,-0.048405,-0.028787,-0.060773,-0.051817,-0.063758,-0.053096,-0.063758,-0.045846,-0.044140,-0.036037,-0.005757,0.005331,-0.000640,-0.001493,0.004904,0.012581,...,0.011728,0.044140,-0.010022,0.084229,0.208333,0.221980,0.149053,-0.000213,0.051390,0.108538,-0.007463,-0.084655,-0.240745,-0.410908,-0.411761,-0.436923,-0.432658,-0.417305,-0.411761,-0.397261,-0.366128,-0.302157,-0.292775,-0.130288,-0.045846,-0.098729,0.052670,0.004052,0.056934,0.125170,0.293201,0.219421,-0.199377,0.298319,0.161421,0.095317,-0.155876,-0.081670,-0.126023,-0.032199
4,0.009596,0.047125,0.021963,0.039449,0.078258,0.131994,0.020684,0.096170,-0.032625,-0.064184,-0.009596,-0.036464,-0.082523,-0.156303,-0.100435,-0.201083,-0.134979,-0.334996,-0.230509,-0.204494,-0.183171,-0.191274,-0.282113,-0.214730,-0.252686,-0.204494,-0.244583,-0.244583,-0.146920,-0.256951,-0.180185,-0.202362,-0.175921,-0.130714,-0.044140,-0.142656,-0.147773,-0.058640,-0.093611,-0.009596,...,0.056508,0.088067,-0.018125,0.033052,0.013860,0.026228,-0.023669,0.009596,-0.028787,-0.008743,-0.078685,0.096596,0.104273,-0.065464,0.182744,-0.041155,0.253113,0.034331,-0.274863,-0.167391,-0.128156,0.160568,0.163979,-0.0

In [ ]:
pp_X_sub_482_rshp_df['sleep_stage']=y_sub_482

In [ ]:
pp_X_sub_482_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,-0.236906,-0.234348,-0.218142,-0.187435,-0.143082,-0.184024,-0.207053,-0.213024,-0.187009,-0.258230,-0.239465,-0.255245,-0.197244,-0.247568,-0.243304,-0.288936,-0.264627,-0.252686,-0.224965,-0.231362,-0.326466,-0.266760,-0.224965,-0.231789,-0.321348,-0.260363,-0.269745,-0.236054,-0.228803,-0.259510,-0.398967,-0.433085,-0.476159,-0.439482,-0.430526,-0.419437,-0.386172,-0.360584,-0.346937,-0.335849,...,-0.044140,-0.023243,-0.036037,-0.033478,0.050537,0.065037,0.020684,0.055655,0.043714,0.129861,0.042008,0.088067,0.100861,0.161847,0.123891,0.052670,0.092332,0.002346,0.081670,0.092758,0.110244,0.033478,0.045420,0.120905,-0.000213,0.012581,0.042008,-0.021537,-0.005331,0.036464,0.134979,0.138817,0.049258,0.100861,0.046272,0.065037,0.008743,-0.012155,0.018125,0
1,0.085082,0.021110,0.123464,0.145215,0.168244,0.133273,0.203641,0.077832,0.107258,0.140097,0.189568,0.091905,0.181891,0.143082,0.151185,0.194686,0.251407,-0.022816,0.083376,0.144362,-0.019405,-0.000213,0.049258,0.088067,0.101714,-0.020684,-0.009169,0.031772,-0.000213,-0.021963,-0.059920,-0.009596,-0.009169,-0.003199,0.030066,0.053096,0.060773,0.017699,0.070155,0.114935,...,0.193406,0.164406,0.147347,0.240745,0.167391,0.169097,0.219421,0.253539,0.282966,0.194259,0.225392,0.224112,0.277422,0.278274,0.297892,0.256951,0.268892,0.321348,0.268039,0.268039,0.404937,0.175494,0.184877,0.080817,-0.077405,-0.132420,-0.193833,-0.084655,-0.110670,-0.075699,0.056934,0.222833,0.234348,0.151185,0.167818,0.214303,0.251833,0.268039,0.275289,0
2,0.242451,0.268466,0.242024,0.272730,0.204494,0.171656,0.211318,0.180185,0.187009,0.182318,0.187862,0.150332,0.161421,0.206200,0.166965,0.145215,0.123891,0.088493,0.026228,0.054375,-0.031346,-0.094038,-0.346084,-0.320922,-0.319216,-0.333716,-0.317084,-0.302584,-0.288936,-0.277422,-0.275289,-0.259083,-0.239039,-0.212171,-0.205347,-0.173362,-0.151185,-0.117920,-0.089773,-0.041581,...,0.037317,0.060773,0.040302,0.039875,0.056508,0.045420,0.048831,0.091905,0.109391,0.122185,0.059067,0.149479,0.115361,0.118347,0.133273,0.118347,0.085082,0.067170,0.071861,0.059067,0.063758,0.052243,-0.002772,-0.008743,-0.019831,0.008743,-0.045846,-0.171656,-0.112376,-0.143509,-0.120905,-0.083376,-0.061626,-0.100861,-0.088493,-0.078258,-0.062905,-0.080817,-0.063331,0
3,-0.134126,-0.107258,-0.096170,-0.097023,-0.117067,-0.099582,-0.099155,-0.100435,-0.083376,-0.098302,-0.118773,-0.105126,-0.098302,-0.044567,-0.040728,-0.050111,-0.065890,-0.069729,-0.038169,-0.020258,-0.083376,-0.065464,-0.065890,-0.008316,-0.048405,-0.028787,-0.060773,-0.051817,-0.063758,-0.053096,-0.063758,-0.045846,-0.044140,-0.036037,-0.005757,0.005331,-0.000640,-0.001493,0.004904,0.012581,...,0.044140,-0.010022,0.084229,0.208333,0.221980,0.149053,-0.000213,0.051390,0.108538,-0.007463,-0.084655,-0.240745,-0.410908,-0.411761,-0.436923,-0.432658,-0.417305,-0.411761,-0.397261,-0.366128,-0.302157,-0.292775,-0.130288,-0.045846,-0.098729,0.052670,0.004052,0.056934,0.125170,0.293201,0.219421,-0.199377,0.298319,0.161421,0.095317,-0.155876,-0.081670,-0.126023,-0.032199,0
4,0.009596,0.047125,0.021963,0.039449,0.078258,0.131994,0.020684,0.096170,-0.032625,-0.064184,-0.009596,-0.036464,-0.082523,-0.156303,-0.100435,-0.201083,-0.134979,-0.334996,-0.230509,-0.204494,-0.183171,-0.191274,-0.282113,-0.214730,-0.252686,-0.204494,-0.244583,-0.244583,-0.146920,-0.256951,-0.180185,-0.202362,-0.175921,-0.130714,-0.044140,-0.142656,-0.147773,-0.058640,-0.093611,-0.009596,...,0.088067,-0.018125,0.033052,0.013860,0.026228,-0.023669,0.009596,-0.028787,-0.008743,-0.078685,0.096596,0.104273,-0.065464,0.182744,-0.041155,0.253113,0.034331,-0.274863,-0.167391,-0.128156,0.160568,0.163979,-0.021963,-0.046699,0.299598,0.2283

In [ ]:
pp_X_sub_482_rshp_df.to_csv(r'/gdrive/My Drive/filtered_x_sub_482.csv', header=False, index=False)

In [ ]:
Fs=100
#batch_size=10

30 epochs

In [ ]:
checkpoint_path_erfan_model = "/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"

In [ ]:
checkpoint_path_erfan_model_dir = os.path.dirname(checkpoint_path_erfan_model)

In [ ]:
cp_callback_erfan_model_30_epochs = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_erfan_model,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=30)

In [ ]:
x_shuf1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
87982,-0.088735,-0.104907,-0.132443,-0.152111,-0.109278,-0.072563,-0.058577,-0.048961,-0.083053,-0.076060,-0.070815,-0.059451,-0.075186,-0.051146,-0.018803,0.028839,0.040640,0.052441,0.024468,0.001740,-0.024485,-0.029730,-0.032789,0.005237,0.029276,0.040203,0.019223,0.004363,-0.006564,-0.028855,-0.020988,-0.034974,-0.038471,-0.010935,0.019223,0.059871,0.088281,0.069050,0.050693,0.046759,...,0.013104,0.023157,0.021409,0.072547,0.100520,0.108387,0.111884,0.083036,0.105327,0.133300,0.156028,0.146413,0.135923,0.099208,0.062931,0.073858,0.106639,0.115380,0.108824,0.107950,0.103142,0.040640,0.072547,0.086970,0.104016,0.112758,0.089593,0.064679,0.055063,0.021409,0.015289,-0.004816,0.005674,-0.010935,-0.013995,-0.031915,-0.002631,0.027965,0.047633,0.040203
68528,0.600975,0.594189,0.584011,0.579345,0.569166,0.521241,0.527603,0.526331,0.537782,0.505549,0.500035,0.498339,0.493249,0.483919,0.459744,0.429208,0.421573,0.389340,0.413939,0.425815,0.408426,0.406305,0.375345,0.324874,0.266770,0.300700,0.310454,0.259984,0.190853,0.226055,0.254047,0.271436,0.213756,0.195094,0.208242,0.251502,0.246413,0.189157,0.169223,0.192974,...,0.516576,0.533965,0.557291,0.568742,0.527179,0.511062,0.558564,0.555171,0.542447,0.526331,0.526331,0.528451,0.504701,0.451686,0.385099,0.380858,0.385099,0.350322,0.286280,0.309606,0.350322,0.315120,0.262529,0.290097,0.261257,0.264226,0.223086,0.249806,0.232417,0.226479,0.208666,0.162437,0.172616,0.149290,0.107302,0.092034,0.062345,0.065738,0.092458,0.064042
37536,-0.034046,-0.019160,-0.031151,0.006478,-0.041076,0.005238,-0.006754,-0.003032,-0.011716,0.016816,0.007305,0.010614,-0.010889,0.016403,0.011027,0.030049,0.006892,-0.007168,-0.019987,0.004411,-0.002619,-0.007995,-0.025362,-0.009649,-0.009649,0.020124,0.012681,-0.001378,0.030462,0.042868,0.008546,0.000689,0.005238,-0.038181,0.003170,-0.010062,0.003170,-0.032806,-0.036527,-0.037354,...,0.134668,0.146247,0.133841,0.145419,0.143765,0.150795,0.145006,0.145833,0.114406,0.108203,0.115646,0.070987,0.105309,0.078844,0.090835,0.079257,0.071400,0.088768,0.061062,0.080498,0.116887,0.025087,0.133841,0.140044,0.113992,0.110684,0.103241,0.088768,0.090422,0.099519,0.088768,0.079671,0.063543,0.043695,0.073881,0.109444,0.090835,0.099106,0.117714,0.104481
49751,-0.057419,0.039738,-0.062918,0.072734,-0.031296,0.052111,-0.072084,0.065401,-0.058335,0.064943,-0.040462,0.048903,-0.022131,0.069984,-0.073458,0.043862,-0.042295,0.050278,-0.071625,0.027364,-0.067501,-0.003799,-0.063834,0.002617,-0.064293,0.010866,-0.053294,-0.001508,-0.044128,0.007199,-0.029922,0.019115,-0.018006,0.011782,-0.036796,0.017282,-0.021672,0.031030,-0.038171,0.014074,...,-0.013423,0.002158,-0.006091,0.001242,-0.037254,-0.014798,-0.007466,0.009033,-0.033130,-0.030380,-0.036337,-0.057419,-0.085832,-0.098664,-0.051919,-0.126161,-0.060627,-0.105538,-0.054669,-0.155033,-0.091790,-0.187571,-0.026255,-0.188487,-0.087665,-0.180238,-0.026714,-0.197653,-0.036337,-0.190779,-0.022131,-0.210943,-0.027172,-0.199945,-0.016631,-0.211860,0.001242,-0.197653,-0.003799,-0.241190
55894,-0.095076,-0.096344,-0.092538,-0.090000,-0.103957,-0.093807,-0.101843,-0.109878,-0.104380,-0.090423,-0.089577,-0.076889,-0.053627,-0.046437,-0.046014,-0.059126,-0.043900,-0.033326,-0.015986,-0.014294,-0.010065,-0.029943,-0.033749,-0.017255,-0.018946,-0.027828,-0.020638,-0.019792,-0.015986,-0.062932,-0.055319,-0.048975,-0.043054,-0.063355,-0.058280,-0.061240,-0.037979,-0.046437,-0.049398,-0.071814,...,-0.063778,-0.054896,-0.070968,-0.065047,-0.063355,-0.058703,-0.065047,-0.046860,-0.044746,-0.042631,-0.024022,-0.005835,0.030538,0.049993,0.017849,0.020387,0.037305,0.032652,0.027577,0.050839,0.045341,0.044918,0.033921,0.052108,0.047455,0.006853,-

In [ ]:
model_30_epochs = model_b(verbose=VBS)
hist_30_epochs = model_30_epochs.fit(
    x_shuf1_rshp, y_shuf1_, batch_size=batch_size, epochs=30, verbose=VBS, callbacks=[cp_callback_erfan_model_30_epochs]
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
y_pred_shuf1 = model_30_epochs.predict(pp_X_sub_482, batch_size=batch_size)
y_pred_shuf1 = np.array([np.argmax(s) for s in y_pred_shuf1])
f1_cnn_shuf1 = f1_score(y_sub_482, y_pred_shuf1, average="macro")
if VBS:
    print("F1 score: {}".format(f1_cnn_shuf1))
    report_shuf1 = classification_report(y_sub_482, y_pred_shuf1)
    print(report_shuf1)

F1 score: 0.7348806164895081
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1351
           1       0.38      0.12      0.19       187
           2       0.76      0.93      0.84       807
           3       0.91      0.84      0.87       318
           4       0.80      0.81      0.81       407

    accuracy                           0.87      3070
   macro avg       0.77      0.73      0.73      3070
weighted avg       0.86      0.87      0.85      3070



30 epochs

F1 score: 0.7348806164895081

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1351
           1       0.38      0.12      0.19       187
           2       0.76      0.93      0.84       807
           3       0.91      0.84      0.87       318
           4       0.80      0.81      0.81       407

    accuracy                           0.87      3070
    macro avg       0.77      0.73      0.73      3070
    weighted avg       0.86      0.87      0.85      3070


  
   


50 subjects

30 epochs: 87%

In [ ]:
latest_30_epochs=tf.train.latest_checkpoint(checkpoint_path_erfan_model_dir)

In [ ]:
print(latest_30_epochs)
epochs_30_model_path="/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"

/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
loaded_model_30_epochs = model_b(verbose=VBS)
loaded_model_30_epochs.load_weights(epochs_30_model_path)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_30_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_sub_482_=np_utils.to_categorical(y_sub_482)

In [ ]:
y_sub_482_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
accuracy_loaded_model_30_epochs = loaded_model_30_epochs.evaluate(pp_X_sub_482, y_sub_482_)

96/96 [==============================] - 2s 5ms/step - loss: 0.4133 - accuracy: 0.8678


In [ ]:
drive.flush_and_unmount()

In [ ]:
from google.colab import drive

drive.mount('gdrive', force_remount=True)

40 epochs

In [ ]:
checkpoint_path_erfan_model_40_epochs = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_40_epochs.ckpt"

In [ ]:
cp_callback_erfan_model_40_epochs = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_erfan_model_40_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=40)

In [ ]:
model_40_epochs = model_b(verbose=VBS)
hist_40_epochs = model_40_epochs.fit(
    x_shuf1_rshp, y_shuf1_, batch_size=batch_size, epochs=40, verbose=VBS, callbacks=[cp_callback_erfan_model_40_epochs]
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
y_pred_shuf1_40_epochs = model_40_epochs.predict(pp_X_sub_482, batch_size=batch_size)
y_pred_shuf1_40_epochs = np.array([np.argmax(s) for s in y_pred_shuf1_40_epochs])
f1_cnn_shuf1 = f1_score(y_sub_482, y_pred_shuf1_40_epochs, average="macro")
if VBS:
    print("F1 score: {}".format(f1_cnn_shuf1))
    report_shuf1 = classification_report(y_sub_482, y_pred_shuf1_40_epochs)
    print(report_shuf1)

F1 score: 0.7459357322832272
              precision    recall  f1-score   support

           0       0.98      0.93      0.96      1351
           1       0.27      0.25      0.26       187
           2       0.79      0.92      0.85       807
           3       0.89      0.93      0.91       318
           4       0.82      0.69      0.75       407

    accuracy                           0.86      3070
   macro avg       0.75      0.74      0.75      3070
weighted avg       0.86      0.86      0.85      3070



50 subjects

40 epochs: 86%

In [ ]:
#print(latest_30_epochs)
epochs_40_model_path="/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_40_epochs.ckpt"

In [ ]:
loaded_model_40_epochs = model_b(verbose=VBS)
loaded_model_40_epochs.load_weights(epochs_40_model_path)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_40_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_loaded_model_40_epochs = loaded_model_40_epochs.evaluate(pp_X_sub_482, y_sub_482_)

96/96 [==============================] - 3s 6ms/step - loss: 0.4413 - accuracy: 0.8554


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 15.4 MB/s 


In [ ]:
#Hyperparameter Tuning

In [ ]:
import keras_tuner as kt

In [ ]:
my_model = model_b(verbose=VBS)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
n_classes=5
use_sub_layer=False
use_rnn=True
verbose=False

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      inputLayer = Input(shape=(3000, 1), name='inLayer')
      convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
      convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
      convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
      convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
      convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
      convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
      convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
      fineShape = convFine.get_shape()
      convFine = Flatten(name='fFlat1')(convFine)
    
      # network to learn coarse features
      convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
      convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
      convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
      convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
      convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
      convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
      convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
      coarseShape = convCoarse.get_shape()
      convCoarse = Flatten(name='cFlat1')(convCoarse)
    
      # concatenate coarse and fine cnns
      mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
      outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
      outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
      outLayer = LSTM(64, return_sequences=True)(outLayer)
      outLayer = LSTM(64, return_sequences=False)(outLayer)

      # Classify
      outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
      model = Model(inputLayer, outLayer)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
      if verbose:
        model.summary()
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_shuf1_rshp,y_shuf1_,
            epochs=hp.Choice("epochs", [10, 20, 30, 40, 50, 60], default=30)
        )

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

In [ ]:
tuner.search(x_shuf1_rshp,y_shuf1_)

Trial 3 Complete [00h 15m 58s]
loss: 0.3501404821872711

Best loss So Far: 0.34688031673431396
Total elapsed time: 00h 43m 04s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
epochs (Choice)
{'default': 30, 'conditions': [], 'values': [10, 20, 30, 40, 50, 60], 'ordered': True}


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 30
Score: 0.34688031673431396
Trial summary
Hyperparameters:
epochs: 30
Score: 0.3501404821872711
Trial summary
Hyperparameters:
epochs: 20
Score: 0.39059850573539734


In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      inputLayer = Input(shape=(3000, 1), name='inLayer')
      convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
      convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
      convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
      convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
      convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
      convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
      convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
      fineShape = convFine.get_shape()
      convFine = Flatten(name='fFlat1')(convFine)
    
      # network to learn coarse features
      convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
      convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
      convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
      convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
      convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
      convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
      convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
      coarseShape = convCoarse.get_shape()
      convCoarse = Flatten(name='cFlat1')(convCoarse)
    
      # concatenate coarse and fine cnns
      mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
      outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
      outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
      outLayer = LSTM(64, return_sequences=True)(outLayer)
      outLayer = LSTM(64, return_sequences=False)(outLayer)

      # Classify
      outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
      model = Model(inputLayer, outLayer)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
      if verbose:
        model.summary()
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_shuf1_rshp,y_shuf1_,
            epochs=hp.Choice("epochs", [10, 20, 30, 40, 50, 60])
        )

In [ ]:
Fs=100

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

In [ ]:
tuner.search(x_shuf1_rshp,y_shuf1_)

Trial 3 Complete [00h 22m 22s]
loss: 0.26555460691452026

Best loss So Far: 0.26555460691452026
Total elapsed time: 00h 40m 11s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 50
Score: 0.26555460691452026
Trial summary
Hyperparameters:
epochs: 30
Score: 0.3435036242008209
Trial summary
Hyperparameters:
epochs: 10
Score: 0.4447125494480133


In [ ]:
#tuner.get_best_hyperparameters()[0]

In [ ]:
#best_model = tuner.get_best_models(1)[0]